# Featurizes the code from Gerrit to make the predictions for verdicts

In [1]:
# This is a python program to create feature vectors from the programming language code
# in such a way that every distinct line is indeed distinct

# author: Miroslaw Staron

import re
import pandas as pd
import time

class FeatureMaker:
    """Class makes a feature vector base on the set of features defined in the class parameters"""

    def __init__(self):
        """The initial state of the feature vector is actually an empty vector"""
        self.featureVector = []

    def addNewFeature(self, newFeature):
        """Adds just one feature to the list of features"""
        self.featureVector.append(newFeature)

    def addNewFeatures(self, newFeatures):
        """Adds a list of features"""
        self.featureVector = self.featureVector + newFeatures
    
    def getFeatureVector(self):
        return self.featureVector

    def featurize(self, line):
        """Counts the frequency of each feature in a given line"""
        self.features = []
        for feature in self.featureVector:
            self.features.append(str(line.count(feature)))
        return self.features
    
    def featuresToString(self):
        strFeatures = '$'.join(str(e) for e in self.features)
        return strFeatures
    
    def findNewFeatures(self, lstTokens):
        newElements = list(set(lstTokens) - set(self.featureVector))
        return newElements

def tokenizeString(myString):
    """This function takes a line and returns a set of strings; empty strings are removed"""
    tokenList = '[\(|"|,|.|;|\)|\[|\]|{|}| ,|\n|\t]'
    tokens = re.split(tokenList, myString)
    tokens = list(filter(None, tokens))
    return tokens

class DataSet:
    """The class makes the connection between the code and its features"""

    def __init__(self):
        self.dictRows = {}
        self.featureVector = []
    
    def addFeatureVector(self, lstFeatureVector):
        self.featureVector = lstFeatureVector

    def addNewLine(self, strLine, lstFeatures):
        self.dictRows[lstFeatures] = strLine 

    def hasLine(self, lstFeatures):
        return (lstFeatures in self.dictRows.keys())
            
    def getLine(self, lstFeatures):
        return self.dictRows[lstFeatures]

    def toCSV(self, strFilename):
        fFile = open(strFilename, 'w', encoding='utf8')
        strFirstLine = 'line$'
        strFirstLine += '$'.join(self.featureVector) + '\n'
        fFile.write(strFirstLine)
        for key, value in self.dictRows.items():
            value = value.replace("\n", "").replace("$","").replace("\r","").replace("\t","")
            strToFile = f'{value}${key}\n'
            fFile.write(strToFile)
        fFile.close()
    
    def flush(self):
        self.dictRows = {}
        self.featureVector = []

def featurizeList(lstLines, strOutputFile):
    dtLines = DataSet()
    
    featurizer = FeatureMaker()

    initialFeatures = tokenizeString(lstLines[0])

    featurizer.addNewFeature(initialFeatures[0])

    start_time = time.time()
    
    foundNewFeature = True
    i = 1
    iLine = 0
    iLastLine = 0
    while foundNewFeature: 
        strTime = f'{(time.time() - start_time):.2f} sec.'
        start_time = time.time()
        foundNewFeature = False
        print(f'Pass number: {i}, time: {strTime}, last processed line: {iLastLine}')
        i += 1
        iLine = 0
        total = len(lstLines)
        for line in lstLines:
            iLine += 1
            if not foundNewFeature: 
                mFeatures = featurizer.featurize(line)  
                if not all(v == 0 for v in mFeatures):    
                    strFeatures = featurizer.featuresToString()
                    if not dtLines.hasLine(strFeatures):
                        dtLines.addNewLine(line, strFeatures)
                    else:
                        strLine = dtLines.getLine(strFeatures)
                        if strLine != line:
                            lineTokens = tokenizeString(line)
                            oldLineTokens = tokenizeString(strLine)
                            newFeatures = featurizer.findNewFeatures(lineTokens+oldLineTokens)
                            if len(newFeatures) > 0:
                                featurizer.addNewFeature(newFeatures[0])
                                foundNewFeature = True
                                dtLines.flush()
                                dtLines.addFeatureVector(featurizer.featureVector)
                                iLastLine = iLine
                                #print(f'Found new feature at line {iLine} of {total}')                                

    return dtLines

def featurizeListMultipleFeatures(lstLines, strOutputFile):
    dtLines = DataSet()
    
    featurizer = FeatureMaker()

    initialFeatures = tokenizeString(lstLines[0])

    featurizer.addNewFeatures(initialFeatures)
    
    start_time = time.time()
    
    foundNewFeature = True
    i = 1
    while foundNewFeature: 
        strTime = f'{(time.time() - start_time)} sec'
        start_time = time.time()
        foundNewFeature = False
        print(f'Pass number: {i}, time: {strTime}, last processed line: {iLine}')
        i += 1
        iLine = 0
        total = len(lstLines)
        for line in lstLines:
            iLine += 1
            if not foundNewFeature: 
                mFeatures = featurizer.featurize(line)  
                if not all(v == 0 for v in mFeatures):    
                    strFeatures = featurizer.featuresToString()
                    if not dtLines.hasLine(strFeatures):
                        dtLines.addNewLine(line, strFeatures)
                    else:
                        strLine = dtLines.getLine(strFeatures)
                        if strLine != line:
                            lineTokens = tokenizeString(line)
                            oldLineTokens = tokenizeString(strLine)
                            newFeatures = featurizer.findNewFeatures(lineTokens+oldLineTokens)
                            if len(newFeatures) > 0:
                                featurizer.addNewFeatures(newFeatures)
                                foundNewFeature = True
                                dtLines.flush()
                                dtLines.addFeatureVector(featurizer.featureVector)
                                #print(f'Found new feature at line {iLine} of {total}')                                

    return dtLines

In [2]:

# this is an recursive function to find a set of features for a set of lines
# the function goes through all lines and then checks which lines are identical given the feature list
# then it takes one of the tokens from the identical lines and makes a recursion
def findFeatureListIterative(lstLines):
    #print(f'Lines: {len(lstLines)}, features: {len(lstFeatures)}')
    
    featurizer = FeatureMaker()
    featureList = []

    if len(lstLines) > 0:
        initialFeatures = tokenizeString(' '.join(lstLines))
        featurizer.addNewFeature(initialFeatures[0])
        featureList.append(initialFeatures[0])
    else:
        return featureList

    featureAdded = True

    while featureAdded:
        dictLinesUnique = {}
        lstNotUnique = []
        featureAdded = False

        # featurizing all lines in this iteration
        for line in lstLines:
            mFeatures = featurizer.featurize(line)
            strFeatures = '$'.join(mFeatures)
            if not (strFeatures in dictLinesUnique.keys()):
                dictLinesUnique[strFeatures] = line
            else:
                lstNotUnique.append(line)
                lstNotUnique.append(dictLinesUnique[strFeatures])
        
        lstNotUnique = list(set(lstNotUnique))

        print(f'Non-unique lines remaining: {len(lstNotUnique)}, features found: {len(featureList)}')

        # and kicking-off the next recursion if necessary
        # by necessary I mean that there are lines that are not different
        if len(lstNotUnique) > 0:
            allLines = lstNotUnique
            getTokens = tokenizeString(' '.join(allLines))
            #getTokens.sort(reverse=True)
            for oneToken in getTokens:
                if not oneToken in featureList:
                    featureList.append(oneToken)
                    featurizer.addNewFeature(oneToken)
                    featureAdded = True
                    break
            if featureAdded:
                #featureList = findFeatureList(allLines, featureList)
                lstLines = lstNotUnique
        
    # returning the feature list
    return featureList

In [3]:
##
## This is a block where we test the featurizer class and the dataset class
##

strOutputFile = 'output_sf_for_verdicts_gromacs.csv'

dfCode = pd.read_csv('./gerrit_reviews_gromacs.csv', 
                    sep=';', 
                    error_bad_lines=False, 
                    warn_bad_lines=True, 
                    header=0, 
                    index_col=False)

mLines = [line for line in dfCode['LOC'] if str(line) != 'nan' ]

print(f'All Lines: {len(mLines)}')

mLines = list(set(mLines))

print(f'Unique Lines: {len(mLines)}')
#print(mLines)



All Lines: 139785
Unique Lines: 40416


In [4]:
features = findFeatureListIterative(mLines)

Non-unique lines remaining: 40416, features found: 1
Non-unique lines remaining: 40416, features found: 2
Non-unique lines remaining: 40414, features found: 3
Non-unique lines remaining: 40414, features found: 4
Non-unique lines remaining: 40413, features found: 5
Non-unique lines remaining: 40412, features found: 6
Non-unique lines remaining: 40411, features found: 7
Non-unique lines remaining: 40411, features found: 8
Non-unique lines remaining: 40410, features found: 9
Non-unique lines remaining: 40410, features found: 10
Non-unique lines remaining: 40408, features found: 11
Non-unique lines remaining: 40408, features found: 12
Non-unique lines remaining: 40408, features found: 13
Non-unique lines remaining: 40407, features found: 14
Non-unique lines remaining: 40401, features found: 15
Non-unique lines remaining: 40388, features found: 16
Non-unique lines remaining: 40370, features found: 17
Non-unique lines remaining: 40368, features found: 18
Non-unique lines remaining: 40362, fe

Non-unique lines remaining: 19813, features found: 152
Non-unique lines remaining: 19813, features found: 153
Non-unique lines remaining: 19812, features found: 154
Non-unique lines remaining: 19812, features found: 155
Non-unique lines remaining: 19619, features found: 156
Non-unique lines remaining: 19609, features found: 157
Non-unique lines remaining: 19593, features found: 158
Non-unique lines remaining: 19589, features found: 159
Non-unique lines remaining: 19574, features found: 160
Non-unique lines remaining: 19567, features found: 161
Non-unique lines remaining: 19567, features found: 162
Non-unique lines remaining: 19567, features found: 163
Non-unique lines remaining: 19526, features found: 164
Non-unique lines remaining: 19525, features found: 165
Non-unique lines remaining: 19525, features found: 166
Non-unique lines remaining: 19525, features found: 167
Non-unique lines remaining: 19512, features found: 168
Non-unique lines remaining: 19488, features found: 169
Non-unique

Non-unique lines remaining: 12317, features found: 301
Non-unique lines remaining: 12316, features found: 302
Non-unique lines remaining: 12315, features found: 303
Non-unique lines remaining: 12314, features found: 304
Non-unique lines remaining: 12002, features found: 305
Non-unique lines remaining: 12002, features found: 306
Non-unique lines remaining: 11996, features found: 307
Non-unique lines remaining: 11959, features found: 308
Non-unique lines remaining: 11958, features found: 309
Non-unique lines remaining: 11947, features found: 310
Non-unique lines remaining: 11947, features found: 311
Non-unique lines remaining: 11947, features found: 312
Non-unique lines remaining: 11946, features found: 313
Non-unique lines remaining: 11908, features found: 314
Non-unique lines remaining: 11905, features found: 315
Non-unique lines remaining: 11905, features found: 316
Non-unique lines remaining: 11902, features found: 317
Non-unique lines remaining: 11901, features found: 318
Non-unique

Non-unique lines remaining: 8699, features found: 452
Non-unique lines remaining: 8699, features found: 453
Non-unique lines remaining: 8699, features found: 454
Non-unique lines remaining: 8699, features found: 455
Non-unique lines remaining: 8699, features found: 456
Non-unique lines remaining: 8697, features found: 457
Non-unique lines remaining: 8697, features found: 458
Non-unique lines remaining: 8693, features found: 459
Non-unique lines remaining: 8693, features found: 460
Non-unique lines remaining: 8680, features found: 461
Non-unique lines remaining: 8680, features found: 462
Non-unique lines remaining: 8517, features found: 463
Non-unique lines remaining: 8517, features found: 464
Non-unique lines remaining: 8517, features found: 465
Non-unique lines remaining: 8517, features found: 466
Non-unique lines remaining: 8513, features found: 467
Non-unique lines remaining: 8513, features found: 468
Non-unique lines remaining: 8513, features found: 469
Non-unique lines remaining: 

Non-unique lines remaining: 7326, features found: 604
Non-unique lines remaining: 7324, features found: 605
Non-unique lines remaining: 7316, features found: 606
Non-unique lines remaining: 7316, features found: 607
Non-unique lines remaining: 7315, features found: 608
Non-unique lines remaining: 7315, features found: 609
Non-unique lines remaining: 7315, features found: 610
Non-unique lines remaining: 7315, features found: 611
Non-unique lines remaining: 7315, features found: 612
Non-unique lines remaining: 7315, features found: 613
Non-unique lines remaining: 7310, features found: 614
Non-unique lines remaining: 7310, features found: 615
Non-unique lines remaining: 7310, features found: 616
Non-unique lines remaining: 7291, features found: 617
Non-unique lines remaining: 7290, features found: 618
Non-unique lines remaining: 7290, features found: 619
Non-unique lines remaining: 7290, features found: 620
Non-unique lines remaining: 7288, features found: 621
Non-unique lines remaining: 

...

Non-unique lines remaining: 4690, features found: 2847
Non-unique lines remaining: 4690, features found: 2848
Non-unique lines remaining: 4690, features found: 2849
Non-unique lines remaining: 4688, features found: 2850
Non-unique lines remaining: 4688, features found: 2851
Non-unique lines remaining: 4688, features found: 2852
Non-unique lines remaining: 4688, features found: 2853
Non-unique lines remaining: 4688, features found: 2854
Non-unique lines remaining: 4688, features found: 2855
Non-unique lines remaining: 4688, features found: 2856
Non-unique lines remaining: 4688, features found: 2857
Non-unique lines remaining: 4688, features found: 2858
Non-unique lines remaining: 4688, features found: 2859
Non-unique lines remaining: 4688, features found: 2860
Non-unique lines remaining: 4688, features found: 2861
Non-unique lines remaining: 4688, features found: 2862
Non-unique lines remaining: 4688, features found: 2863
Non-unique lines remaining: 4688, features found: 2864
Non-unique

Non-unique lines remaining: 4671, features found: 2996
Non-unique lines remaining: 4671, features found: 2997
Non-unique lines remaining: 4671, features found: 2998
Non-unique lines remaining: 4671, features found: 2999
Non-unique lines remaining: 4671, features found: 3000
Non-unique lines remaining: 4671, features found: 3001
Non-unique lines remaining: 4671, features found: 3002
Non-unique lines remaining: 4671, features found: 3003
Non-unique lines remaining: 4671, features found: 3004
Non-unique lines remaining: 4671, features found: 3005
Non-unique lines remaining: 4671, features found: 3006
Non-unique lines remaining: 4671, features found: 3007
Non-unique lines remaining: 4671, features found: 3008
Non-unique lines remaining: 4671, features found: 3009
Non-unique lines remaining: 4671, features found: 3010
Non-unique lines remaining: 4671, features found: 3011
Non-unique lines remaining: 4671, features found: 3012
Non-unique lines remaining: 4671, features found: 3013
Non-unique

Non-unique lines remaining: 4653, features found: 3145
Non-unique lines remaining: 4653, features found: 3146
Non-unique lines remaining: 4653, features found: 3147
Non-unique lines remaining: 4653, features found: 3148
Non-unique lines remaining: 4653, features found: 3149
Non-unique lines remaining: 4653, features found: 3150
Non-unique lines remaining: 4653, features found: 3151
Non-unique lines remaining: 4653, features found: 3152
Non-unique lines remaining: 4653, features found: 3153
Non-unique lines remaining: 4653, features found: 3154
Non-unique lines remaining: 4653, features found: 3155
Non-unique lines remaining: 4653, features found: 3156
Non-unique lines remaining: 4653, features found: 3157
Non-unique lines remaining: 4653, features found: 3158
Non-unique lines remaining: 4653, features found: 3159
Non-unique lines remaining: 4653, features found: 3160
Non-unique lines remaining: 4653, features found: 3161
Non-unique lines remaining: 4653, features found: 3162
Non-unique

Non-unique lines remaining: 4640, features found: 3294
Non-unique lines remaining: 4640, features found: 3295
Non-unique lines remaining: 4640, features found: 3296
Non-unique lines remaining: 4640, features found: 3297
Non-unique lines remaining: 4638, features found: 3298
Non-unique lines remaining: 4638, features found: 3299
Non-unique lines remaining: 4638, features found: 3300
Non-unique lines remaining: 4638, features found: 3301
Non-unique lines remaining: 4638, features found: 3302
Non-unique lines remaining: 4638, features found: 3303
Non-unique lines remaining: 4638, features found: 3304
Non-unique lines remaining: 4637, features found: 3305
Non-unique lines remaining: 4636, features found: 3306
Non-unique lines remaining: 4636, features found: 3307
Non-unique lines remaining: 4636, features found: 3308
Non-unique lines remaining: 4636, features found: 3309
Non-unique lines remaining: 4636, features found: 3310
Non-unique lines remaining: 4636, features found: 3311
Non-unique

Non-unique lines remaining: 4608, features found: 3443
Non-unique lines remaining: 4608, features found: 3444
Non-unique lines remaining: 4607, features found: 3445
Non-unique lines remaining: 4607, features found: 3446
Non-unique lines remaining: 4607, features found: 3447
Non-unique lines remaining: 4605, features found: 3448
Non-unique lines remaining: 4603, features found: 3449


In [8]:
def featurizeListPredefined(lstLines, lstFeatures):
    dtLines = DataSet()
    
    dtLines.addFeatureVector(lstFeatures)
    
    featurizer = FeatureMaker()

    featurizer.addNewFeatures(lstFeatures)

    foundNewFeature = True
    i = 1
    while foundNewFeature:
        foundNewFeature = False
        print(f'Pass number: {i}')
        i += 1
        iLine = 0
        total = len(lstLines)
        for line in lstLines:
            iLine += 1
            if not foundNewFeature: 
                mFeatures = featurizer.featurize(line)  
                if not all(v == 0 for v in mFeatures):    
                    strFeatures = featurizer.featuresToString()
                    if not dtLines.hasLine(strFeatures):
                        dtLines.addNewLine(line, strFeatures)
                    else:
                        strLine = dtLines.getLine(strFeatures)
                        if strLine != line:
                            lineTokens = tokenizeString(line)
                            oldLineTokens = tokenizeString(strLine)
                            newFeatures = featurizer.findNewFeatures(lineTokens+oldLineTokens)
                            if len(newFeatures) > 0:
                                featurizer.addNewFeature(newFeatures[0])
                                foundNewFeature = True
                                dtLines.flush()
                                dtLines.addFeatureVector(featurizer.featureVector)
                                print(f'Found new feature at line {iLine} of {total}')                           

    return dtLines

In [9]:
print(len(features))

3449


In [10]:
strOutputFile = './featurized_lines.csv'
dtLines = featurizeListPredefined(mLines, features)
dtLines.toCSV(strOutputFile)

Pass number: 1
